# Scaler Transformations
___

The following notebook includes the following steps:

- Train-Test Split
- KNN Imputation
- Intialize function for creating a dictionary for Sci-kit Learn scalers and their transformations on the data

In [23]:
# Split Data between train and test

X_train = df.loc['12-9200':'12-9300',num_cols]
y_train = df.loc['12-9200':'12-9300',"Ground Truth Anomaly"]
X_test = df.loc['12-9301':'12-9370',num_cols]
y_test = df.loc['12-9301':'12-9370',"Ground Truth Anomaly"]


# X_train = df.loc['12-9263':'12-9325',num_cols]
# y_train = df.loc['12-9263':'12-9325',"Ground Truth Anomaly"]
# X_test = df.loc['12-9326':'12-9388',num_cols]
# y_test = df.loc['12-9326':'12-9388',"Ground Truth Anomaly"]

In [24]:
print()
print(y_train.value_counts())



0.0    96
1.0     1
Name: Ground Truth Anomaly, dtype: int64


In [25]:
y_test.value_counts()

0.0    63
1.0     8
Name: Ground Truth Anomaly, dtype: int64

In [26]:
# Use KNN Imputer for missing values
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
imputed_train = imputer.fit_transform(X_train)
X_train_imputed = pd.DataFrame(imputed_train, columns=X_train.columns)
X_train_imputed.index = X_train.index


imputed_test = imputer.fit_transform(X_test)
X_test_imputed = pd.DataFrame(imputed_test, columns=X_test.columns)
X_test_imputed.index = X_test.index

## Transformation Function
___

The following function is not of my own creation and all credit goes to Kefei Mo (https://towardsdatascience.com/pca-a-practical-journey-preprocessing-encoding-and-inspiring-applications-64371cb134a)

In [2]:
def transformer_bundle(X_train, is_full_compare=True, is_get_instance=False):
    if is_full_compare:
        transformer_dict = {'StandardScaler': StandardScaler(), 
                            'QuantileTransformer':QuantileTransformer(), 
                            'RobustScaler':RobustScaler(), 
                            #'Normalizer':Normalizer(), 
                            'MaxAbsScaler':MaxAbsScaler(), 
                            'MinMaxScaler':MinMaxScaler(), 
                            'PowerTransformer':PowerTransformer()}
    else:
        transformer_dict = {'StandardScaler': StandardScaler(), 
                            'QuantileTransformer':QuantileTransformer(), 
                            'PowerTransformer':PowerTransformer()}
    
    X_train_trans_dict = {trs:transformer_dict[trs].fit_transform(X_train) for trs in transformer_dict.keys()} 
    X_train_trans_dict['NoTransformer'] = X_train.values
    
    X_train_pca_dict = {trs:PCA().fit_transform(X_train_trans_dict[trs]) for trs in transformer_dict.keys()}
    X_train_pca_dict['NoTransformer'] = PCA().fit_transform(X_train)
    
    instance_pca_dict = {trs:PCA().fit(X_train_trans_dict[trs]) for trs in transformer_dict.keys()}
    instance_pca_dict['NoTransformer'] = PCA().fit(X_train)
    
    if not is_get_instance:
        return X_train_trans_dict, X_train_pca_dict
    else:
        return X_train_trans_dict, X_train_pca_dict, instance_pca_dict